In [1]:
import pandas as pd
import numpy as np

In [2]:
import featuretools as ft

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
clients = pd.read_csv('./data/clients.csv', parse_dates = ['joined'])
loans = pd.read_csv('./data/loans.csv', parse_dates = ['loan_start', 'loan_end'])
payments = pd.read_csv('./data/payments.csv', parse_dates = ['payment_date'])

In [5]:
clients.head()

,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527
1,49545,2007-11-14,104564,770
2,41480,2013-03-11,122607,585
3,46180,2001-11-06,43851,562
4,25707,2006-10-06,211422,621


In [6]:
loans.sample(10)

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
235,32961,credit,12553,1,11113,2001-09-08,2003-06-15,2.98
238,32961,other,13674,0,11932,2013-11-24,2016-08-08,2.15
265,44601,home,8635,1,10700,2004-03-08,2006-10-07,4.97
150,35089,home,10947,0,11793,2000-09-08,2003-01-14,0.67
365,26695,other,9323,1,10468,2002-10-22,2005-03-17,2.79
77,46180,credit,5362,0,10237,2013-01-07,2014-05-23,2.48
175,35214,other,4953,1,10908,2010-07-23,2012-05-02,0.16
243,32961,credit,10602,1,11427,2008-06-05,2009-12-19,4.44
140,35089,home,9175,1,11547,2003-04-02,2005-04-21,5.36
51,41480,other,4491,1,10511,2002-06-08,2004-08-09,2.82


In [7]:
payments.sample(10)

,loan_id,payment_amount,payment_date,missed
1877,10144,1400,2002-02-09,1
1159,10336,1791,2014-10-26,0
2679,10683,1019,2007-05-03,1
307,10888,2719,2004-06-10,0
1611,10763,1261,2007-11-09,0
2180,10681,1788,2009-01-29,1
1320,11968,848,2014-10-24,1
2902,11197,1659,2007-08-12,1
3246,10714,1057,2003-10-26,1
2232,10024,878,2006-01-11,1


In [8]:
clients['join_month'] = clients['joined'].dt.month
clients['log_income'] = np.log(clients['income'])
clients.head()

,client_id,joined,income,credit_score,join_month,log_income
0,46109,2002-04-16,172677,527,4,12.059178
1,49545,2007-11-14,104564,770,11,11.557555
2,41480,2013-03-11,122607,585,3,11.716739
3,46180,2001-11-06,43851,562,11,10.688553
4,25707,2006-10-06,211422,621,10,12.261611


In [9]:
stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 'max', 'min'])
stats.columns = ['mean_loan_amount', 'max_loan_amount', 'min_loan_amount']
stats.head()

,mean_loan_amount,max_loan_amount,min_loan_amount
client_id,,,
25707,7963.950000,13913,1212
26326,7270.062500,13464,1164
26695,7824.722222,14865,2389
26945,7125.933333,14593,653
29841,9813.000000,14837,2778


In [10]:
clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left').head(10)

,client_id,joined,income,credit_score,join_month,log_income,mean_loan_amount,max_loan_amount,min_loan_amount
0,46109,2002-04-16,172677,527,4,12.059178,8951.600000,14049,559
1,49545,2007-11-14,104564,770,11,11.557555,10289.300000,14971,3851
2,41480,2013-03-11,122607,585,3,11.716739,7894.850000,14399,811
3,46180,2001-11-06,43851,562,11,10.688553,7700.850000,14081,1607
4,25707,2006-10-06,211422,621,10,12.261611,7963.950000,13913,1212
5,39505,2011-10-14,153873,610,10,11.943883,7424.050000,14575,904
6,32726,2006-05-01,235705,730,5,12.370336,6633.263158,14802,851
7,35089,2010-03-01,131176,771,3,11.784295,6939.200000,13194,773
8,35214,2003-08-08,95849,696,8,11.470529,7173.555556,14767,667
9,48177,2008-06-09,190632,769,6,12.158100,7424.368421,14740,659


In [11]:
es = ft.EntitySet(id = 'clients')

In [29]:
et = ft.EntitySet(id = 'clients')

In [12]:
es = es.entity_from_dataframe(entity_id = 'clients', dataframe = clients, 
                              index = 'client_id', time_index = 'joined')

In [ ]:
et = 

In [13]:
es = es.entity_from_dataframe(entity_id = 'loans', dataframe = loans, 
                              variable_types = {'repaid': ft.variable_types.Categorical},
                              index = 'loan_id', 
                              time_index = 'loan_start')

In [14]:
es = es.entity_from_dataframe(entity_id = 'payments', 
                              dataframe = payments,
                              variable_types = {'missed': ft.variable_types.Categorical},
                              make_index = True,
                              index = 'payment_id',
                              time_index = 'payment_date')


In [15]:
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

In [16]:
es['loans']

Entity: loans
  Variables:
    loan_id (dtype: index)
    client_id (dtype: numeric)
    loan_type (dtype: categorical)
    loan_amount (dtype: numeric)
    loan_start (dtype: datetime_time_index)
    loan_end (dtype: datetime)
    rate (dtype: numeric)
    repaid (dtype: categorical)
  Shape:
    (Rows: 443, Columns: 8)

In [17]:
# Relationship between clients and previous loans
r_client_previous = ft.Relationship(es['clients']['client_id'],
                                    es['loans']['client_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_client_previous)

In [18]:
r_payments = ft.Relationship(es['loans']['loan_id'],
                                      es['payments']['loan_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_payments)

es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

In [22]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(10)

,name,type,description
0,sum,aggregation,Sums elements of a numeric or boolean feature.
1,median,aggregation,Finds the median value of any feature with well-ordered values.
2,all,aggregation,Test if all values are 'True'.
3,num_unique,aggregation,Returns the number of unique categorical variables.
4,count,aggregation,Counts the number of non null values.
5,avg_time_between,aggregation,Computes the average time between consecutive events.
6,any,aggregation,Test if any value is 'True'.
7,max,aggregation,Finds the maximum non-null value of a numeric feature.
8,n_most_common,aggregation,Finds the N most common elements in a categorical feature.
9,min,aggregation,Finds the minimum non-null value of a numeric feature.


In [23]:
primitives[primitives['type'] == 'transform'].head(10)

,name,type,description
20,modulo_by_feature,transform,
21,add_numeric_scalar,transform,
22,greater_than_equal_to,transform,
23,cum_count,transform,Returns the cumulative count after grouping
24,week,transform,Transform a Datetime feature into the week.
25,modulo_numeric,transform,
26,not_equal_scalar,transform,
27,greater_than,transform,
28,longitude,transform,Returns the second value on the tuple base feature.
29,second,transform,Transform a Datetime feature into the second.


In [27]:
# Create new features using specified primitives
features, feature_names = ft.dfs(entityset = es, target_entity = 'clients', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last'],
                                 trans_primitives = ['year', 'month', 'divide_numeric'])

In [28]:
features

,income,credit_score,join_month,log_income,MEAN(loans.loan_amount),MEAN(loans.rate),MAX(loans.loan_amount),MAX(loans.rate),LAST(loans.loan_type),LAST(loans.loan_amount),...,MEAN(loans.loan_amount) / LAST(loans.rate),MAX(payments.payment_amount) / income,LAST(loans.rate) / MEAN(loans.loan_amount),LAST(payments.payment_amount) / credit_score,credit_score / MAX(payments.payment_amount),credit_score / MEAN(loans.rate),MEAN(loans.rate) / MEAN(payments.payment_amount),MAX(loans.loan_amount) / log_income,LAST(loans.loan_amount) / income,MAX(payments.payment_amount) / log_income
client_id,,,,,,,,,,,,,,,,,,,,,
25707,211422,621,10,12.261611,7963.950000,3.477000,13913,9.44,home,2203,...,1076.209459,0.012790,0.000929,0.388084,0.229660,178.602243,0.002950,1134.679573,0.010420,220.525664
26326,227920,633,5,12.336750,7270.062500,2.517500,13464,6.73,credit,5275,...,5013.836207,0.011662,0.000199,1.472354,0.238149,251.439921,0.002158,1091.373338,0.023144,215.453828
26695,174532,680,8,12.069863,7824.722222,2.466111,14865,6.51,other,13918,...,8694.135802,0.016799,0.000115,3.064706,0.231924,275.737779,0.002042,1231.579806,0.079745,242.919071
26945,214516,806,11,12.276140,7125.933333,2.855333,14593,5.65,cash,9249,...,2491.585082,0.012903,0.000401,1.981390,0.291185,282.278777,0.002574,1188.728743,0.043116,225.478048
29841,38354,523,8,10.554614,9813.000000,3.445000,14837,6.76,home,7223,...,1927.897839,0.075559,0.000519,1.529637,0.180469,151.814224,0.002393,1405.735904,0.188325,274.571858
32726,235705,730,5,12.370336,6633.263158,3.058947,14802,9.10,other,5325,...,2343.909243,0.011434,0.000427,1.387671,0.270872,238.644184,0.003239,1196.572158,0.022592,217.859881
32885,58955,642,5,10.984530,9920.400000,2.436000,14162,9.11,other,11886,...,1088.957190,0.041082,0.000918,2.973520,0.265070,263.546798,0.001744,1289.267758,0.201611,220.491916
32961,230341,714,4,12.347316,7882.235294,3.930588,14784,9.14,cash,1693,...,5398.791297,0.012117,0.000185,0.312325,0.255822,181.652200,0.003783,1197.345227,0.007350,226.041026
35089,131176,771,3,11.784295,6939.200000,3.513500,13194,7.63,other,773,...,909.462647,0.019539,0.001100,0.137484,0.300819,219.439306,0.003209,1119.625719,0.005893,217.492854
